<a href="https://colab.research.google.com/github/ericphamhung-gretel/demos/blob/main/data_designer_sdk_qa_pairs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📚 Data Designer SDK: Generate Diverse Q&A Pairs

This notebook demonstrates a general approach for extracting Q&A pairs from any source document
(e.g. text, markdown, or PDF files). The generated Q&A pairs can be used for:
- **Instruction Tuning:** Training models with clear, self-contained examples.
- **Retrieval-Augmented Generation (RAG):** Enhancing retrieval systems with precise and context-supported Q&A pairs.
- **Search and FAQ Systems:** Powering natural language query systems and documentation.

> **Note:** The [Data Designer](https://docs.gretel.ai/create-synthetic-data/gretel-data-designer-beta) functionality demonstrated in this notebook is currently in **Early Preview**. To access these features and run this notebook, please [join the waitlist](https://gretel.ai/navigator/data-designer#waitlist).

# 📘 Getting Started

First, let's install and import the required packages:

In [1]:
# Install required libraries

%%capture
!pip install -qq langchain unstructured[pdf] smart_open git+https://github.com/gretelai/gretel-python-client

In [22]:
# Configuration
# -------------
# Define your source document(s) and the number of Q&A pairs to generate.
# You can replace this with your own documents in PDF, markdown, or text formats.

DOCUMENT_LIST = ["https://www.americanexpress.com/content/dam/amex/us/rewards/membership-rewards/mr-terms-conditions-11.01.2024.pdf?mrlinknav=footer-tandc"]
NUM_QA_PAIRS = 100

In [3]:
# Document Processing
# ------------------
# The DocumentProcessor class handles loading and chunking source documents for RAG evaluation.
# We use langchain's RecursiveCharacterTextSplitter and unstructured.io for robust document parsing.

from typing import List, Union
from langchain.text_splitter import RecursiveCharacterTextSplitter
from unstructured.partition.auto import partition
from smart_open import open
import tempfile
import os

class DocumentProcessor:
    def __init__(self, chunk_size: int = 4192, chunk_overlap: int = 200):
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            length_function=len,
        )

    def parse_document(self, uri: str) -> str:
        """Parse a single document from URI into raw text."""
        with open(uri, 'rb') as file:
            content = file.read()
            with tempfile.NamedTemporaryFile(delete=False) as temp_file:
                temp_file.write(content)
                temp_file.flush()
                elements = partition(temp_file.name)

        os.unlink(temp_file.name)
        return "\n\n".join([str(element) for element in elements])

    def process_documents(self, uris: Union[str, List[str]]) -> List[str]:
        """Process one or more documents into chunks for RAG evaluation."""
        if isinstance(uris, str):
            uris = [uris]

        all_chunks = []
        for uri in uris:
            text = self.parse_document(uri)
            chunks = self.text_splitter.split_text(text)
            all_chunks.extend(chunks)

        return all_chunks

In [4]:
# Data Models
# -----------
# Define Pydantic models for structured output generation:
# 1. QAPair: Schema for question-answer pairs
# 2. EvalMetrics: Schema for scoring generation quality

from pydantic import BaseModel, Field
from typing import Optional, Literal

class QAPair(BaseModel):
    question: str = Field(..., description="A clear and concise question derived from the context.")
    answer: str = Field(..., description="A detailed and accurate answer fully supported by the context.")
    reasoning: str = Field(..., description="A brief explanation of why this Q&A pair is valuable.")


class EvalMetrics(BaseModel):
    clarity: int = Field(..., description="Clarity of the question", ge=1, le=5)
    factual_accuracy: int = Field(..., description="Factual accuracy of the answer", ge=1, le=5)
    comprehensiveness: int = Field(..., description="Completeness of the answer", ge=1, le=5)
    reasoning_quality: int = Field(..., description="Quality of the provided reasoning", ge=1, le=5)

In [23]:
# Setup & Configure Data Designer
# --------------------------------
# Initialize the Data Designer with a custom system message to ensure that the generated
# Q&A pairs are high-quality, context-supported, and tailored for a variety of downstream applications.


from gretel_client.navigator import DataDesigner

# Process document chunks
processor = DocumentProcessor(chunk_size=4192, chunk_overlap=200)
chunks = processor.process_documents(DOCUMENT_LIST)

# Initialize Data Designer
designer = DataDesigner(
    api_key="prompt",
    model_suite="llama-3.x",  # or "apache-2.0" as needed
    endpoint="https://api.gretel.cloud",
    special_system_instructions="""\
You are an expert at generating high-quality, context-supported Q&A pairs.
Your output should be clear, concise, and factually correct.
Ensure that every question is self-contained and every answer is comprehensive and derived solely from the provided context.
"""
)

# Add Seed Columns for Controlled Diversity

# Context: Document chunks.
designer.add_categorical_seed_column(
    name="context",
    description="A chunk of text extracted from the source document.",
    values=chunks
)

# Difficulty with nested Sophistication.
designer.add_categorical_seed_column(
    name="difficulty",
    description="The overall difficulty level of the question.",
    values=["easy", "medium", "hard"],
    subcategories=[
        {
            "name": "sophistication",
            "values": {
                "easy": ["basic", "straightforward"],
                "medium": ["intermediate", "moderately complex"],
                "hard": ["advanced", "sophisticated"]
            }
        }
    ]
)

# Question Style: Tone and approach.
designer.add_categorical_seed_column(
    name="question_style",
    description="The style or tone of the question.",
    values=["factual", "exploratory", "analytical", "comparative"]
)

# Target Audience: Language complexity.
designer.add_categorical_seed_column(
    name="target_audience",
    description="The intended audience for the Q&A pair.",
    values=["novice", "intermediate", "expert"]
)

# Response Format: Expected answer style.
designer.add_categorical_seed_column(
    name="response_format",
    description="The format of the answer expected (e.g., short, detailed, step-by-step).",
    values=["short", "detailed", "step-by-step", "list"]
)

# Define Generation Template for Q&A Pairs
designer.add_generated_data_column(
    name="qa_pair",
    generation_prompt=(
        "\n{context}\n\n"
        "Based on the above context, generate a high-quality Q&A pair. The question should be clear and concise, "
        "and tailored for an audience at the '{target_audience}' level. The overall difficulty should be '{difficulty}', "
        "with a corresponding sophistication level (e.g., {sophistication}). The question style should be '{question_style}', "
        "and the answer should be provided in a '{response_format}' format. "
        "Ensure that the answer is factually accurate, fully derived from the context, and comprehensive. "
        "Additionally, include a brief explanation (reasoning) on why this Q&A pair is valuable.\n\n"
        "Output the result in JSON format as follows:\n"
        '{{\n'
        '  "question": "<your question>",\n'
        '  "answer": "<your answer>",\n'
        '  "reasoning": "<your explanation>"\n'
        '}}\n'
    ),
    data_config={
        "type": "structured",
        "params": {"model": QAPair}
    }
)

# Optional: Define Evaluation Template for Q&A Pairs
designer.add_generated_data_column(
    name="eval_metrics",
    llm_type="judge",
    generation_prompt=(
        "\n{context}\n\n"
        "For the above Q&A pair:\n"
        "{qa_pair}\n\n"
        "Evaluate the following criteria on a scale of 1 (lowest) to 5 (highest):\n"
        "1. Clarity: How clear and understandable is the question?\n"
        "2. Factual Accuracy: Is the answer fully supported by the context?\n"
        "3. Comprehensiveness: Does the answer cover all necessary details?\n"
        "4. Reasoning Quality: Is the provided reasoning well-justified and clear?\n\n"
        "Output the scores in JSON format."
    ),
    data_config={
        "type": "structured",
        "params": {"model": EvalMetrics}
    }
)

# Preview a Sample of 10 Generated Records
preview = designer.generate_dataset_preview()
# preview.display_sample_record()


[22:18:05] [INFO] 🦜 Using llama-3.x model suite
Gretel API Key: ··········
Logged in as eric.phamhung@gretel.ai ✅
[22:18:12] [INFO] 🚀 Generating dataset preview
[22:18:13] [INFO] 📥 Step 1: Load data seeds
[22:18:13] [INFO] 🎲 Step 2: Sample data seeds
[22:18:13] [INFO] 🦜 Step 3: Generate column from template >> generating qa pair
[22:18:27] [INFO] 🦜 Step 4: Generate column from template >> generating eval metrics
[22:18:42] [INFO] 👀 Your dataset preview is ready for a peek!


In [24]:
from rich.console import Console
from rich.table import Table
from collections import Counter
import pandas as pd

def generate_qa_report(df):
    console = Console()
    categories = ['difficulty', 'sophistication', 'question_style', 'target_audience', 'response_format']

    console.print("\n[bold blue]🧠 QA Pair Generation Report[/bold blue]", justify="center")
    console.print("=" * 80, justify="center")
    console.print(f"\n[bold]Total QA Pairs:[/bold] {len(df)}")

    for category in categories:
        if category in df.columns:
            # Only count non-empty values
            counts = Counter(x for x in df[category] if pd.notna(x) and x != '')
            if not counts:
                continue

            table = Table(title=f"\n{category.title()} Distribution")
            table.add_column("Category", style="cyan")
            table.add_column("Count", justify="right")
            table.add_column("Percentage", justify="right")

            total = sum(counts.values())
            for value, count in sorted(counts.items()):
                percentage = (count / total) * 100
                table.add_row(str(value), str(count), f"{percentage:.1f}%")

            console.print(table)

    if any(col.startswith('eval_metrics.') for col in df.columns):
        metrics_table = Table(title="\nQuality Metrics Summary")
        metrics_table.add_column("Metric")
        metrics_table.add_column("Average Score", justify="right")

        metrics = ['clarity', 'factual_accuracy', 'comprehensiveness', 'reasoning_quality']
        for metric in metrics:
            col = f'eval_metrics.{metric}'
            if col in df.columns:
                scores = df[col].dropna()
                if len(scores) > 0:
                    avg_score = scores.mean()
                    metrics_table.add_row(
                        metric.replace('_', ' ').title(),
                        f"{avg_score:.2f}/5.00"
                    )

        console.print(metrics_table)

generate_qa_report(preview.dataset)

🧠 QA Pair Generation Report

================================================================================

Total QA Pairs: 10

                                 
     Difficulty Distribution     
┏━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━┓
┃ Category ┃ Count ┃ Percentage ┃
┡━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━┩
│ easy     │     2 │      20.0% │
│ hard     │     5 │      50.0% │
│ medium   │     3 │      30.0% │
└──────────┴───────┴────────────┘

                                           
        Sophistication Distribution        
┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━┓
┃ Category           ┃ Count ┃ Percentage ┃
┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━┩
│ advanced           │     4 │      40.0% │
│ intermediate       │     1 │      10.0% │
│ moderately complex │     2 │      20.0% │
│ sophisticated      │     1 │      10.0% │
│ straightforward    │     2 │      20.0% │
└────────────────────┴───────┴────────────┘

                                    
    Question_Style Distribution     
┏━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━┓
┃ Category    ┃ Count ┃ Percentage ┃
┡━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━┩
│ analytical  │     6 │      60.0% │
│ comparative │     1 │      10.0% │
│ exploratory │     2 │      20.0% │
│ factual     │     1 │      10.0% │
└─────────────┴───────┴────────────┘

                                     
    Target_Audience Distribution     
┏━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━┓
┃ Category     ┃ Count ┃ Percentage ┃
┡━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━┩
│ expert       │     5 │      50.0% │
│ intermediate │     1 │      10.0% │
│ novice       │     4 │      40.0% │
└──────────────┴───────┴────────────┘

                                     
    Response_Format Distribution     
┏━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━┓
┃ Category     ┃ Count ┃ Percentage ┃
┡━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━┩
│ detailed     │     2 │      20.0% │
│ list         │     2 │      20.0% │
│ short        │     3 │      30.0% │
│ step-by-step │     3 │      30.0% │
└──────────────┴───────┴────────────┘

In [25]:
# View the preview dataset as a dataframe
preview.dataset

,context,difficulty,sophistication,question_style,target_audience,response_format,qa_pair,eval_metrics
0,11\n\nLimitation on Arbitration\n\nArbitration...,medium,moderately complex,analytical,expert,step-by-step,"{""question"": ""What are the specific conditions...","{""clarity"": 5, ""factual_accuracy"": 5, ""compreh..."
1,11\n\nLimitation on Arbitration\n\nArbitration...,medium,intermediate,exploratory,novice,list,"{""question"": ""What are the limitations on arbi...","{""clarity"": 5, ""factual_accuracy"": 5, ""compreh..."
2,American Express ADR c/o CT Corporation System...,hard,advanced,exploratory,intermediate,step-by-step,"{""question"": ""What are the limitations on arbi...","{""clarity"": 5, ""factual_accuracy"": 5, ""compreh..."
3,1 additional point (for a total of 2 points) o...,hard,advanced,analytical,expert,list,"{""question"": ""What are the specific purchase c...","{""clarity"": 5, ""factual_accuracy"": 5, ""compreh..."
4,4 additional points (for a total of 5 points) ...,easy,straightforward,comparative,expert,step-by-step,"{""question"": ""Which American Express card offe...","{""clarity"": 5, ""factual_accuracy"": 5, ""compreh..."
5,4 additional points (for a total of 5 points) ...,easy,straightforward,analytical,novice,detailed,"{""question"": ""What is the limit for earning ad...","{""clarity"": 5, ""factual_accuracy"": 5, ""compreh..."
6,Use Redeem for Deposits for a deposit into you...,hard,sophisticated,analytical,novice,short,"{""question"": ""What are the conditions to be el...","{""clarity"": 5, ""factual_accuracy"": 2, ""compreh..."
7,arbitration organization and all parties in wr...,hard,advanced,analytical,novice,detailed,"{""question"": ""If you disagree with an arbitrat...","{""clarity"": 5, ""factual_accuracy"": 5, ""compreh..."
8,If you don't make a timely payment of the mini...,medium,moderately complex,factual,expert,short,"{""question"": ""What is the fee for requesting p...","{""clarity"": 4, ""factual_accuracy"": 5, ""compreh..."
9,Business Purchase Account\n\nNo\n\n$40\n\nNo\n...,hard,advanced,analytical,expert,short,"{""question"": ""What is the condition for a Corp...","{""clarity"": 5, ""factual_accuracy"": 5, ""compreh..."


In [34]:
preview

PreviewResults(output=                                             context difficulty  \
0  11\n\nLimitation on Arbitration\n\nArbitration...     medium   
1  11\n\nLimitation on Arbitration\n\nArbitration...     medium   
2  American Express ADR c/o CT Corporation System...       hard   
3  1 additional point (for a total of 2 points) o...       hard   
4  4 additional points (for a total of 5 points) ...       easy   
5  4 additional points (for a total of 5 points) ...       easy   
6  Use Redeem for Deposits for a deposit into you...       hard   
7  arbitration organization and all parties in wr...       hard   
8  If you don't make a timely payment of the mini...     medium   
9  Business Purchase Account\n\nNo\n\n$40\n\nNo\n...       hard   

       sophistication question_style target_audience response_format  \
0  moderately complex     analytical          expert    step-by-step   
1        intermediate    exploratory          novice            list   
2            advanced   

In [26]:
preview.display_sample_record(3)

                                             Categorical Seed Columns                                              
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Name                ┃ Value                                                                                     ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ context             │ 1 additional point (for a total of 2 points) on gasoline at gas stations in the U.S. (not │
│                     │ including superstores, supermarkets or warehouse clubs that sell gasoline). 50% extra     │
│                     │ points - make 30 or more separate purchases of goods or services in a billing period and  │
│                     │ earn 50% extra points on those purchases (less returns and credits). A transaction where  │
│                     │ you buy multiple items counts as one purchase. Purchases do not include cash advances,    │
│                     │ balance transfers, purchases of travelers cheques, purchases or reloading of prepaid      │
│                     │ cards, or purchases of other cash equivalents. Does not apply to limited time promotions  │
│                     │ and offers (such as a Welcome Bonus). Returns and credits do not reduce your purchase     │
│                     │ count but will reduce the number of points and extra points earned. (Example: Suppose in  │
│                     │ a billing period you make 30 purchases totaling $500 and you return a $200 item that you  │
│                     │ purchased in the same or a prior billing period. The return does not reduce your purchase │
│                     │ count but it does reduce the points on those purchases from 500 to 300. So, you would     │
│                     │ earn 50% extra points on the 300 points.) To count purchases, we use the purchase date.   │
│                     │ In rare instances, a purchase may not post to your account for several billing periods.   │
│                     │ If a purchase posts to your account more than 4 billing periods after the purchase date,  │
│                     │ we will count it in the billing period in which it posts. Some merchants combine separate │
│                     │ charges into a single transaction. When this happens, the combined charges will count as  │
│                     │ a single purchase for the purpose of counting purchases. The purchase date may not be the │
│                     │ date you made a transaction - for example, if you buy goods online, the purchase date may │
│                     │ be the date the goods are shipped. The 50% extra points are typically awarded 6 to 8      │
│                     │ weeks after the billing period in which you make the 30 or more eligible purchases.       │
│                     │                                                                                           │
│                     │ 2 additional points (for a total of 3 points) for each dollar charged on eligible travel  │
│                     │ purchases including airfare, hotels, cruises, car rentals, campgrounds, trains, taxicabs, │
│                     │ rideshare services, tours, ferries, tolls, parking, buses, subways, on third party travel │
│                     │ websites, and on amextravel.com. You will not earn additional points for purchases of     │
│                     │ timeshare properties. 2 additional points (for a total of 3 points) for each dollar       │
│                     │ charged at restaurants worldwide.                                                         │
│                     │                                                                                           │
│                     │ No                                                                                        │
│                     │                                 

In [27]:
df = preview.dataset
df

,context,difficulty,sophistication,question_style,target_audience,response_format,qa_pair,eval_metrics
0,11\n\nLimitation on Arbitration\n\nArbitration...,medium,moderately complex,analytical,expert,step-by-step,"{""question"": ""What are the specific conditions...","{""clarity"": 5, ""factual_accuracy"": 5, ""compreh..."
1,11\n\nLimitation on Arbitration\n\nArbitration...,medium,intermediate,exploratory,novice,list,"{""question"": ""What are the limitations on arbi...","{""clarity"": 5, ""factual_accuracy"": 5, ""compreh..."
2,American Express ADR c/o CT Corporation System...,hard,advanced,exploratory,intermediate,step-by-step,"{""question"": ""What are the limitations on arbi...","{""clarity"": 5, ""factual_accuracy"": 5, ""compreh..."
3,1 additional point (for a total of 2 points) o...,hard,advanced,analytical,expert,list,"{""question"": ""What are the specific purchase c...","{""clarity"": 5, ""factual_accuracy"": 5, ""compreh..."
4,4 additional points (for a total of 5 points) ...,easy,straightforward,comparative,expert,step-by-step,"{""question"": ""Which American Express card offe...","{""clarity"": 5, ""factual_accuracy"": 5, ""compreh..."
5,4 additional points (for a total of 5 points) ...,easy,straightforward,analytical,novice,detailed,"{""question"": ""What is the limit for earning ad...","{""clarity"": 5, ""factual_accuracy"": 5, ""compreh..."
6,Use Redeem for Deposits for a deposit into you...,hard,sophisticated,analytical,novice,short,"{""question"": ""What are the conditions to be el...","{""clarity"": 5, ""factual_accuracy"": 2, ""compreh..."
7,arbitration organization and all parties in wr...,hard,advanced,analytical,novice,detailed,"{""question"": ""If you disagree with an arbitrat...","{""clarity"": 5, ""factual_accuracy"": 5, ""compreh..."
8,If you don't make a timely payment of the mini...,medium,moderately complex,factual,expert,short,"{""question"": ""What is the fee for requesting p...","{""clarity"": 4, ""factual_accuracy"": 5, ""compreh..."
9,Business Purchase Account\n\nNo\n\n$40\n\nNo\n...,hard,advanced,analytical,expert,short,"{""question"": ""What is the condition for a Corp...","{""clarity"": 5, ""factual_accuracy"": 5, ""compreh..."


In [35]:
# # Generate and Analyze Dataset
# # ---------------------------
# # Uncomment these lines to generate evaluation pairs, save them to JSONL,
# # and analyze the coverage and quality of the generated dataset.

batch_job = designer.submit_batch_workflow(num_records=NUM_QA_PAIRS)
dataset = batch_job.fetch_dataset(wait_for_completion=True)

generate_qa_report(dataset)

dataset.to_json('qa_pairs.jsonl', orient='records', lines=True)


[22:23:37] [INFO] ⚙️ Configuring Data Designer Workflow steps:
[22:23:37] [INFO]   |-- Step 1: load-data-seeds-1
[22:23:37] [INFO]   |-- Step 2: sample-data-seeds-2
[22:23:37] [INFO]   |-- Step 3: generate-column-from-template-3-generating-qa-pair
[22:23:37] [INFO]   |-- Step 4: generate-column-from-template-4-generating-eval-metrics
[22:23:38] [INFO] 🛜 Connecting to your Gretel Project:
[22:23:38] [INFO] 🔗 -> https://console.gretel.ai/proj_2sujzltw68y7xjAQ0it4eoSGlTo
[22:23:39] [INFO] ▶️ Starting your workflow run to generate 100 records:
[22:23:39] [INFO]   |-- project_name: gretel-sdk-4932a-44ab09eab0fa0bb
[22:23:39] [INFO]   |-- project_id: proj_2sujzltw68y7xjAQ0it4eoSGlTo
[22:23:39] [INFO]   |-- workflow_run_id: wr_2sujzzvx7cpGCIKg1FVR3ZMfptn
[22:23:39] [INFO] 🔗 -> https://console.gretel.ai/workflows/w_2sujzww0db2uK7m0Emnpjxbm4eE/runs/wr_2sujzzvx7cpGCIKg1FVR3ZMfptn
[22:23:42] [INFO] ⏳ Waiting for workflow step `generate-column-from-template-4-generating-eval-metrics` to complete..

🧠 QA Pair Generation Report

================================================================================

Total QA Pairs: 100

                                 
     Difficulty Distribution     
┏━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━┓
┃ Category ┃ Count ┃ Percentage ┃
┡━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━┩
│ easy     │    35 │      35.0% │
│ hard     │    34 │      34.0% │
│ medium   │    31 │      31.0% │
└──────────┴───────┴────────────┘

                                           
        Sophistication Distribution        
┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━┓
┃ Category           ┃ Count ┃ Percentage ┃
┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━┩
│ advanced           │    19 │      19.0% │
│ basic              │    17 │      17.0% │
│ intermediate       │    15 │      15.0% │
│ moderately complex │    16 │      16.0% │
│ sophisticated      │    15 │      15.0% │
│ straightforward    │    18 │      18.0% │
└────────────────────┴───────┴────────────┘

                                    
    Question_Style Distribution     
┏━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━┓
┃ Category    ┃ Count ┃ Percentage ┃
┡━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━┩
│ analytical  │    19 │      19.0% │
│ comparative │    26 │      26.0% │
│ exploratory │    33 │      33.0% │
│ factual     │    22 │      22.0% │
└─────────────┴───────┴────────────┘

                                     
    Target_Audience Distribution     
┏━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━┓
┃ Category     ┃ Count ┃ Percentage ┃
┡━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━┩
│ expert       │    36 │      36.0% │
│ intermediate │    33 │      33.0% │
│ novice       │    31 │      31.0% │
└──────────────┴───────┴────────────┘

                                     
    Response_Format Distribution     
┏━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━┓
┃ Category     ┃ Count ┃ Percentage ┃
┡━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━┩
│ detailed     │    27 │      27.0% │
│ list         │    29 │      29.0% │
│ short        │    25 │      25.0% │
│ step-by-step │    19 │      19.0% │
└──────────────┴───────┴────────────┘

In [36]:
dataset

,context,difficulty,sophistication,question_style,target_audience,response_format,qa_pair,eval_metrics
0,Lose Points\n\nThis section of the Terms and C...,medium,intermediate,factual,novice,step-by-step,"{""question"": ""What happens to the points in my...","{""clarity"": 5, ""comprehensiveness"": 5, ""factua..."
1,1 additional point (for a total of 2 points) o...,easy,straightforward,factual,novice,short,"{""question"": ""How many points do you earn for ...","{""clarity"": 5, ""factual_accuracy"": 5, ""compreh..."
2,If an airline loyalty program stops participat...,medium,intermediate,comparative,novice,step-by-step,"{""question"": ""How does linking a partner loyal...","{""clarity"": 5, ""factual_accuracy"": 5, ""compreh..."
3,arbitration organization and all parties in wr...,easy,basic,factual,intermediate,short,"{""question"": ""If you disagree with the arbitra...","{""clarity"": 5, ""comprehensiveness"": 5, ""factua..."
4,3. Gasoline at gas stations in the U.S. (not i...,hard,advanced,factual,novice,step-by-step,"{""question"": ""How do you determine the top 2 c...","{""clarity"": 5, ""comprehensiveness"": 5, ""factua..."
...,...,...,...,...,...,...,...,...
95,If you don't make a timely payment of the mini...,hard,advanced,analytical,intermediate,list,"{""question"": ""What are the conditions under wh...","{""clarity"": 4, ""comprehensiveness"": 4, ""factua..."
96,The list of eligible charges can change from t...,hard,sophisticated,analytical,expert,list,"{""question"": ""What are the specific limitation...","{""clarity"": 5, ""comprehensiveness"": 5, ""factua..."
97,Arbitration procedures are generally simpler t...,hard,sophisticated,exploratory,intermediate,step-by-step,"{""question"": ""What are the limitations on arbi...","{""clarity"": 4, ""factual_accuracy"": 4, ""compreh..."
98,4 additional points (for a total of 5 points) ...,medium,moderately complex,exploratory,expert,detailed,"{""question"": ""How do the rewards programs on A...","{""clarity"": 5, ""comprehensiveness"": 4, ""factua..."


# Using OpenAI

In [29]:
from openai import OpenAI
from google.colab import userdata


In [30]:
client = OpenAI(
    base_url="https://litellm-proxy.dev.gretel.cloud/v1",
    api_key=userdata.get("GRETEL_API_KEY")
)

# [m.id for m in client.models.list()]

In [31]:
system_prompt = """
You are an expert at generating high-quality, context-supported Q&A pairs.
Your output should be clear, concise, and factually correct.
Ensure that every question is self-contained and every answer is comprehensive and derived solely from the provided context.
"""

user_prompt = f'''

"\n{chunks}\n\n"
"Based on the above context, generate {NUM_QA_PAIRS} high-quality Q&A pair. The question should be clear and concise, "
"Ensure that the answer is factually accurate, fully derived from the context, and comprehensive. "
"Additionally, include a brief explanation (reasoning) on why this Q&A pair is valuable.\n\n"
"Output the result in JSON format as follows:\n"
"{{\n"
"  "question": "<your question>",\n"
"  "answer": "<your answer>",\n"
"  "reasoning": "<your explanation>"\n"
"}}\n"


'''

In [32]:
completion = client.chat.completions.create(
    model='gpt-4o',
    messages=[
        {"role": "system", "content": system_prompt},
        {
            "role": "user",
            "content": user_prompt
        }
    ]
)
print(completion.choices[0].message)




ChatCompletionMessage(content='```json\n[\n  {\n    "question": "What is the name of the program described in the document?",\n    "answer": "Membership Rewards® Program",\n    "reasoning": "Identifying the program is fundamental to understanding the context and scope of the provided terms and conditions."\n  },\n  {\n    "question": "Who is the owner and operator of the Membership Rewards Program?",\n    "answer": "American Express Travel Related Services Company, Inc.",\n    "reasoning": "Knowing the program operator helps to understand who manages the program and can also be useful for addressing any customer service issues."\n  },\n  {\n    "question": "When was the document last updated?",\n    "answer": "November 2024",\n    "reasoning": "The update date is crucial for determining the currency and relevancy of the provided information."\n  },\n  {\n    "question": "What might American Express do to the Membership Rewards Program according to the document?",\n    "answer": "Americ

In [33]:
print(completion.choices[0].message.content)

```json
[
  {
    "question": "What is the name of the program described in the document?",
    "answer": "Membership Rewards® Program",
    "reasoning": "Identifying the program is fundamental to understanding the context and scope of the provided terms and conditions."
  },
  {
    "question": "Who is the owner and operator of the Membership Rewards Program?",
    "answer": "American Express Travel Related Services Company, Inc.",
    "reasoning": "Knowing the program operator helps to understand who manages the program and can also be useful for addressing any customer service issues."
  },
  {
    "question": "When was the document last updated?",
    "answer": "November 2024",
    "reasoning": "The update date is crucial for determining the currency and relevancy of the provided information."
  },
  {
    "question": "What might American Express do to the Membership Rewards Program according to the document?",
    "answer": "American Express may make changes to the Rewards Program